<a href="https://colab.research.google.com/github/jubeam/my_js_project/blob/master/CNN_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from urllib.request import urlretrieve
urlretrieve('http://doc.mindscale.kr/km/unstructured/dog-vs-cat.zip',
            'dog-vs-cat.zip')
from zipfile import ZipFile

with ZipFile('dog-vs-cat.zip') as z:
    z.extractall()


from keras.preprocessing.image import ImageDataGenerator

img_gen = ImageDataGenerator(
        rescale=1/255,         # 픽셀 값을 0~1 범위로 변환
        rotation_range=40,      # 40도까지 회전
        width_shift_range=0.2,  # 20%까지 좌우 이동
        height_shift_range=0.2, # 20%까지 상하 이동
        shear_range=0.2,        # 20%까지 기울임
        zoom_range=0.2,         # 20%까지 확대
        horizontal_flip=True,   # 좌우 뒤집기
    )

train = img_gen.flow_from_directory(
    'dog-vs-cat/train',     # 이미지 디렉토리
    target_size=(100, 100), # 변환할 크기는 가로 100, 세로 100
    color_mode='rgb',       # 컬러는 rgb, 흑백은 grayscale. 생략하면 컬러로 처리한다
    class_mode='binary')    # 고양이 vs. 개로 binary 분류

valid = ImageDataGenerator(rescale=1/255).flow_from_directory(
    'dog-vs-cat/validation',
    target_size=(100, 100),
    class_mode='binary',
    shuffle=False)

#모형 만들기
from keras.layers import Conv2D, Dense, Dropout, Flatten, MaxPooling2D
from keras.models import Sequential

model = Sequential()
model.add(Conv2D(32, (5, 5), activation='relu', input_shape=(100, 100, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.summary()

from keras.optimizers import Adam, RMSprop
from keras.callbacks import TensorBoard
LOG_DIR = 'log_model'
model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=Adam())
from keras.callbacks import ModelCheckpoint, TensorBoard
model.fit_generator(
    train, validation_data=valid,steps_per_epoch=10, validation_steps=5, epochs=5,
    callbacks=[
        TensorBoard(log_dir=LOG_DIR),
        ModelCheckpoint('cnn-{epoch:02d}.hdf5', save_best_only=True),
    ])

from keras.preprocessing.image import load_img 
img = load_img('dog-vs-cat/validation/cat/cat.1000.jpg', target_size=(100, 100))
import numpy
img_array = numpy.array(img) / 255
img_array.shape
model.predict(numpy.array([img_array]))


!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip
get_ipython().system_raw(f'tensorboard --logdir {LOG_DIR} --host 0.0.0.0 --port 6006 &')
get_ipython().system_raw('./ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | tr '"' '\n' | grep "https://"

Using TensorFlow backend.
W0704 06:37:43.695105 140511775905664 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0704 06:37:43.738786 140511775905664 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0704 06:37:43.745626 140511775905664 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


W0704 06:37:43.793528 140511775905664 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0704 06:37:43.826372 140511775905664 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0704 06:37:43.838260 140511775905664 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0704 06:37:43.887930 140511775905664 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.p

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 96, 96, 32)        2432      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 48, 48, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 46, 46, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 23, 23, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 16928)             0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16928)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 16929     
Total para

W0704 06:37:47.835361 140511775905664 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/callbacks.py:850: The name tf.summary.merge_all is deprecated. Please use tf.compat.v1.summary.merge_all instead.

W0704 06:37:47.836921 140511775905664 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/callbacks.py:853: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.



Epoch 1/5
10/10 [==============================] - 8s 829ms/step - loss: 0.7881 - acc: 0.4813 - val_loss: 0.5877 - val_acc: 0.9313
Epoch 2/5
10/10 [==============================] - 6s 581ms/step - loss: 0.7205 - acc: 0.5438 - val_loss: 1.1116 - val_acc: 0.0000e+00
Epoch 3/5
10/10 [==============================] - 6s 586ms/step - loss: 0.7049 - acc: 0.5344 - val_loss: 0.6907 - val_acc: 0.5062
Epoch 4/5
10/10 [==============================] - 6s 579ms/step - loss: 0.6908 - acc: 0.5250 - val_loss: 0.6151 - val_acc: 1.0000
Epoch 5/5
10/10 [==============================] - 6s 580ms/step - loss: 0.6944 - acc: 0.5156 - val_loss: 0.7804 - val_acc: 0.0312
--2019-07-04 06:38:20--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.71.139.107, 52.203.53.176, 54.165.51.142, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.71.139.107|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17556757 (17M) [applic